In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
%matplotlib inline
from keras.preprocessing import image
import keras.backend as K
import os
import glob
import numpy as np
!pip install facenet-pytorch
from PIL import Image
from facenet_pytorch.models.mtcnn import MTCNN
from google.colab.patches import cv2_imshow
from random import randint
import random
import zipfile
import cv2
import dlib
import torch
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math
import albumentations as A
import os

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 7.7 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
celebA_path = "/content/drive/MyDrive/Omdena DeepFake Image Detection (preprocessed data)/Raw Data/0-CelebA"
ffhq_path = "/content/drive/MyDrive/Omdena DeepFake Image Detection (preprocessed data)/Raw Data/0-FFHQ"
attgan_path = "/content/drive/MyDrive/Omdena DeepFake Image Detection (preprocessed data)/Raw Data/1-ATTGAN"
gdwct_path = "/content/drive/MyDrive/Omdena DeepFake Image Detection (preprocessed data)/Raw Data/1-GDWCT"
stylegan_path = "/content/drive/MyDrive/Omdena DeepFake Image Detection (preprocessed data)/Raw Data/1-STYLEGAN"
stylegan2_path = "/content/drive/MyDrive/Omdena DeepFake Image Detection (preprocessed data)/Raw Data/1-STYLEGAN2/1-STYLEGAN2"
stargan_path = "/content/drive/MyDrive/Omdena DeepFake Image Detection (preprocessed data)/Raw Data/1-StarGAN"

In [ ]:
#celebA_aug_path = "/content/aug/celebA_aug"
ffhq_aug_path = "/content/aug/ffhq_aug"
attgan_aug_path = "/content/aug/attgan_aug"
gdwct_aug_path = "/content/aug/gdwct_path"
stylegan_aug_path = "/content/aug/stylegan_path"
stylegan2_aug_path = "/content/aug/stylegan2_path"
stargan_aug_path = "/content/aug/stargan_path"

In [ ]:
def blackout_eyes(img, boxes, landmarks):
  left_eye_x, left_eye_y = landmarks[0][0][0], landmarks[0][0][1]
  right_eye_x, right_eye_y = landmarks[0][1][0], landmarks[0][1][1]

  box_width, box_height = boxes[0][2]*0.8, boxes[0][3]*0.1              
  #defining width and height of black rectangle to be created
  width_margin, height_margin = 0.1*box_width, box_height*0.4
  ## old--->> width_margin, height_margin = abs(left_eye_x - right_eye_x)*0.25, 10
  #width_margin, height_margin = width*0.8, box_height*0.1
  if(landmarks[0][0][1] >= landmarks[0][1][1]):
    start = (math.ceil(left_eye_x - width_margin), math.ceil(left_eye_y - height_margin))
    end = (math.ceil(right_eye_x + width_margin), math.ceil(right_eye_y + height_margin))
  else:
    end = (math.ceil(left_eye_x - width_margin), math.ceil(left_eye_y - height_margin))
    start = (math.ceil(right_eye_x + width_margin), math.ceil(right_eye_y + height_margin))
  cv2.rectangle(img, start, end, (0,0,0),-1)
  return img

def blackout_nose(img, boxes, landmarks):
  nose_x, nose_y = landmarks[0][2][0], landmarks[0][2][1]
  box_width, box_height = boxes[0][2]*0.8, boxes[0][3]*0.1
  width_margin, height_margin = 0.15*box_width, box_height*0.5
  start = (math.ceil(nose_x - width_margin), math.ceil(nose_y - height_margin*1.5))
  end = (math.ceil(nose_x + width_margin), math.ceil(nose_y + height_margin))
  cv2.rectangle(img, start, end, (0,0,0),-1)
  return img

def blackout_mouth(img, boxes, landmarks):
  left_mouth_x, left_mouth_y = landmarks[0][3][0], landmarks[0][3][1]
  right_mouth_x, right_mouth_y = landmarks[0][4][0], landmarks[0][4][1]
  box_width, box_height = boxes[0][2]*0.5, boxes[0][3]*0.15
  #defining width and height of black rectangle to be created
  width_margin, height_margin = 0.1*box_width, box_height*0.35
  ## old--->> width_margin, height_margin = abs(left_eye_x - right_eye_x)*0.25, 10
  #width_margin, height_margin = width*0.8, box_height*0.1
  start = (math.ceil(left_mouth_x - width_margin), math.ceil(left_mouth_y - height_margin))
  end = (math.ceil(right_mouth_x + width_margin), math.ceil(right_mouth_y + height_margin))
  cv2.rectangle(img, start, end, (0,0,0),-1)
  return img 

In [ ]:
def blackout(img, boxes, probs, landmarks):
  if not probs[0]:     #incase no face detected by MTCNN
    return img

  if probs[0] < 0.95:  #in case MTCNN not sure about exact face landmarks, can be due to blurring
    return img

  m = randint(0, 3)  ##Random number
  
  if m==0:
    return blackout_eyes(img, boxes, landmarks)
  
  elif m==1:
    return blackout_nose(img, boxes, landmarks)

  else:
    return blackout_mouth(img, boxes, landmarks)

In [ ]:
def mtcnn_params(img):
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    mtcnn = MTCNN(keep_all=True, device=device)
    boxes, probs, landmarks = mtcnn.detect(img, landmarks=True)
    return boxes, probs, landmarks

In [ ]:
transform = A.Compose([
    A.ImageCompression(70, 100, p=0.8)
], p=1)

In [ ]:
def preprocess_img(img):               #--------no changes required---------#
    # resize the image to (160, 160)
    img = cv2.resize(img, (160, 160))
    # add Gaussian noise with mean=0 and std=10
    noise = np.random.normal(loc=0, scale=8, size=img.shape)
    img = np.clip(img + noise, 0, 255).astype(np.uint8)
    # add image compression
    #-----------------------CHANGED----------------------------------#
    img = transform(image = img)["image"]  #using albumentations
    return img

####Run this code for different folders, only folder name and (3/6) parameter needs to be changed

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# Define the ImageDataGenerator object with the desired augmentations  
datagen1 = ImageDataGenerator(     # 1st datagen obj
    rotation_range=360,
    horizontal_flip=True,
    shear_range=0.2,
    width_shift_range=0.2,  
    fill_mode = "reflect",  
)

datagen2 = ImageDataGenerator(     #2nd datagen obj
    rotation_range=360,
    vertical_flip=True,
    zoom_range=0.2,
    height_shift_range=0.2,
    fill_mode = "constant",
)

# Define the path from which we want to read the images
# path = celebA_path + "/"  # TODO Saurav                #-----CHANGE ACCORDINGLY--------#
#path = ffhq_path + "/"  # TODO Saurav
#path = attgan_path + "/"
# path = gdwct_path + "/"
# path = stylegan2_path + "/"
path = stylegan2_path + "/"
#path = stargan_path + "/"

# Get a list of all files (images) in that folder
files = os.listdir(path)

# Initialize the count variable for naming of augmented images while saving
COUNT = 0
os.makedirs("/content/stylegan2_aug")       #-----CHANGE ACCORDINGLY--------#  -> may give error incase folder already created
files = os.listdir(path)
for file in files:
  img = cv2.imread(path + file) 
  #------------------IMPORTANT-------------------------#
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) #Without this images apprearing in distorted color

  boxes, probs, landmarks = mtcnn_params(img)  # getting landmarks of face
  new_img = blackout(img, boxes, probs, landmarks)  # applying blackout on the image
  resized_img = preprocess_img(new_img)
  resized_img = resized_img.reshape((1, ) + resized_img.shape)

  # Genrating a random number -> 1 or 2 to ensure 50% probability
  m = random.randint(1, 3)
                            
  if m==1:
    datagen = datagen1
  else:
    datagen = datagen2

  i = 0
  for _ in datagen.flow(resized_img, batch_size = 1, 
                        save_to_dir = "/content/stylegan2_aug",   #-----CHANGE ACCORDINGLY--------#
                        save_prefix = f"stylegan2_aug_{COUNT}",           #-----CHANGE ACCORDINGLY--------#
                        save_format = "jpg"):
    i+=1                                                                   
    COUNT+=1  #to keep track of total number of images
    if i==6:  #for real, limit = 3;  deepfake, limit = 6      #-----CHANGE ACCORDINGLY--------#
      break 


print(f"COUNT should be 6000, check -> {COUNT}")              #-----CHANGE ACCORDINGLY--------#

COUNT should be 6000, check -> 6000


In [ ]:
imgs = os.listdir("/content/stylegan2_aug")
print(len(imgs))

5998


###I have added code of other file here itself

##Splitting the files into train, val and test
###**First create 3 more folders**
1. celebA_Aug_Train
2. celebA_Aug_Val
3. celebA_Aug_Test

In [ ]:
import os
import numpy as np
import shutil

# # Creating Train / Val / Test folders (One time use)
root_dir = '/content/stylegan2_aug'         #-----------change accordingly
train_dir = "/content/stylegan2_Aug_Train"  #-----------change accordingly
val_dir = "/content/stylegan2_Aug_Val"      #-----------change accordingly
test_dir = "/content/stylegan2_Aug_Test"    #-----------change accordingly

!rm -rf `find -type d -name .ipynb_checkpoints`

allFileNames = os.listdir(root_dir)
np.random.shuffle(allFileNames)

train_FileNames, val_FileNames, test_FileNames = np.split(np.array(allFileNames),
                                                          [int(len(allFileNames)*0.8), int(len(allFileNames)*0.9)])


train_FileNames = [root_dir+'/'+ name for name in train_FileNames.tolist()]
val_FileNames = [root_dir+'/' + name for name in val_FileNames.tolist()]
test_FileNames = [root_dir+'/' + name for name in test_FileNames.tolist()]


print('Total images: ', len(allFileNames))
print('Training: ', len(train_FileNames))
print('Validation: ', len(val_FileNames))
print('Testing: ', len(test_FileNames))

#Copy-pasting images
for name in train_FileNames:
    shutil.copy(name, train_dir)

for name in val_FileNames:
    shutil.copy(name, val_dir)

for name in test_FileNames:
    shutil.copy(name, test_dir)

Total images:  5998
Training:  4798
Validation:  600
Testing:  600


###Creating Zip folder of all 3 folders created before
Can't download folder directly, so we have to create zip folder of each

In [ ]:
from google.colab import files
import os

# Format for below line -->
# !zip -r /content/final_Folder_name.zip /content/Folder_to_be_zipped

#change accordingly
!zip -r /content/stylegan2_Aug_Train.zip /content/stylegan2_Aug_Train  #zipping train folder   #---Change accordingly-----#
!zip -r /content/stylegan2_Aug_Val.zip /content/stylegan2_Aug_Val      #zipping val folder     #---Change accordingly-----#
!zip -r /content/stylegan2_Aug_Test.zip /content/stylegan2_Aug_Test    #zipping test folder    #---Change accordingly-----#

Streaming output truncated to the last 5000 lines.
  adding: content/stylegan2_Aug_Train/stylegan2_aug_2772_0_7688.jpg (deflated 3%)
  adding: content/stylegan2_Aug_Train/stylegan2_aug_2550_0_1997.jpg (deflated 2%)
  adding: content/stylegan2_Aug_Train/stylegan2_aug_300_0_3172.jpg (deflated 2%)
  adding: content/stylegan2_Aug_Train/stylegan2_aug_918_0_6264.jpg (deflated 3%)
  adding: content/stylegan2_Aug_Train/stylegan2_aug_1038_0_867.jpg (deflated 3%)
  adding: content/stylegan2_Aug_Train/stylegan2_aug_5784_0_8191.jpg (deflated 3%)
  adding: content/stylegan2_Aug_Train/stylegan2_aug_96_0_5332.jpg (deflated 3%)
  adding: content/stylegan2_Aug_Train/stylegan2_aug_1164_0_6299.jpg (deflated 2%)
  adding: content/stylegan2_Aug_Train/stylegan2_aug_4440_0_7099.jpg (deflated 3%)
  adding: content/stylegan2_Aug_Train/stylegan2_aug_5388_0_4864.jpg (deflated 2%)
  adding: content/stylegan2_Aug_Train/stylegan2_aug_5286_0_3818.jpg (deflated 2%)
  adding: content/stylegan2_Aug_Train/stylegan2_aug_

##Done!
Now download the 3 zip files (train, test, val) on your computer